In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Install fairseq
import os

# If POS integration with input embedding
os.chdir('.........../POS-implementation/fairseq/Fairseq-POS-integration-input-embedding')

# If POS integration with positional encoding
os.chdir('.........../POS-implementation/fairseq/Fairseq-POS-integration-positional-encoding')

!pip install --editable .
!pip install sentencepiece sacrebleu tensorboardX

In [ ]:
import os
os.chdir('...........src/Subword-segmentation/Transformer-BPE')

In [ ]:
# Preprocess the data
os.chdir('............../scripts')
!bash preprocess-joined-ensi.sh

In [ ]:
import os  
os.chdir("............./Transformer-BPE")
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/en_si_bpe5000/ \
    --source-lang en --target-lang si \
    --arch transformer \
    --encoder-layers 5 --decoder-layers 5 \
    --encoder-num-pos 42 \
    --encoder-pos-embed-dim 16 \
    --combine-bpe-path 'path to combine.bpe.en file' \
    --combine-numbers-path 'path to combine.numbers.en file' \
    --encoder-embed-dim 512 --decoder-embed-dim 512 \
    --encoder-normalize-before --decoder-normalize-before \
    --encoder-ffn-embed-dim 2048 --decoder-ffn-embed-dim 2048 \
    --encoder-attention-heads 2 --decoder-attention-heads 2 \
    --dropout 0.4 --attention-dropout 0.2 --relu-dropout 0.2 \
    --weight-decay 0.0001 \
    --label-smoothing 0.2 --criterion label_smoothed_cross_entropy \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0 \
    --lr-scheduler inverse_sqrt --warmup-updates 4000 --warmup-init-lr 1e-7 \
    --lr 1e-3 --min-lr 1e-9 \
    --batch-size 32 \
    --update-freq 4 \
    --max-epoch 200 --save-interval 10 \
    --tensorboard-logdir logs/

In [ ]:
#200 epoch - validation set
# Obtain the BLEU score for validation dataset
!fairseq-generate \
    data-bin/en_si_bpe5000/ \
    --source-lang en --target-lang si \
    --path checkpoints/checkpoint_best.pt \
    --beam 5 --lenpen 1.2 \
    --gen-subset valid \
    --remove-bpe=sentencepiece

In [ ]:
#200 epoch - test set
# Obtain the BLEU score for test dataset
!fairseq-generate \
    data-bin/en_si_bpe5000/ \
    --source-lang en --target-lang si \
    --path checkpoints/checkpoint_best.pt \
    --beam 5 --lenpen 1.2 \
    --gen-subset test \
    --remove-bpe=sentencepiece

In [ ]:
# Obtain the checkpoint ensemble BLEU score for validation dataset
!fairseq-generate \
    data-bin/en_si_bpe5000/ \
    --source-lang en --target-lang si \
    --path checkpoints/checkpoint_last.pt:checkpoints/checkpoint190.pt \
    --beam 5 --lenpen 1.2 \
    --gen-subset valid \
    --remove-bpe=sentencepiece

In [ ]:
# Obtain the checkpoint ensembled BLEU score for test dataset
!fairseq-generate \
    data-bin/en_si_bpe5000/ \
    --source-lang en --target-lang si \
    --path checkpoints/checkpoint_last.pt:checkpoints/checkpoint190.pt \
    --beam 5 --lenpen 1.2 \
    --gen-subset test \
    --remove-bpe=sentencepiece

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Visualize using tensorboard
%tensorboard --logdir logs/